In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import functools

from subset_helper import cleaner_climate

**<font color='blue'> Notebook description</font>**

- <font color='blue'>This script cleans and featurizes the raw meteorological data of different cities in Belgium, taking the reference period into account. Finally, the mortality data and the meteorological data are combined into a single data file for predictive modeling. </font>

- <font color='blue'>This script also featurizes the raw meteorological data from different European countries. As the data cleaning procedures are similar, functions are wrapped in the class *subset_helper*.</font>

In [2]:
# Import raw data - Belgium mortality 
mortality = pd.read_excel('../data/cause of death.xlsx') # Monthly mortality data with causes from Stabel
reference_dist = pd.read_csv('../data/reference_dist_1983_to_2008.csv') # 1983 Jan 01 - 2008 Dec 31 as references

In [3]:
# First 16 rows are text / sentences
ROW_SKIP = 16
climate_brussels = pd.read_csv('../data/Daily_20090101_20201231_brussels.csv', skiprows = ROW_SKIP)
climate_dinant = pd.read_csv('../data/Daily_20090101_20201231_dinant.csv',  skiprows = ROW_SKIP)
climate_antwerp = pd.read_csv('../data/Daily_20090101_20201231_antwerp.csv', skiprows = ROW_SKIP)

In [4]:
# Import raw data - Multiple countries
reference_dist_multicountries = pd.read_csv('../data/reference_europe_1983_to_2008.csv') # 1983 Jan 01 - 2008 Dec 31 as references

In [5]:
ROW_SKIP = 11
def read_file(filename, country):
    data = pd.read_csv('../data/multi_countries/' + filename + '.csv', skiprows = ROW_SKIP)
    data['COUNTRY'] = country 
    return data

In [6]:
filenames = ['Belgium', 'France', 'Greece', 'Russia', 'Spain', 'UK', 'Romania']
df_countries = []

for filename in filenames:
    data = read_file(filename, filename)
    df_countries.append(data)

In [7]:
# Check column names
mortality.columns

Index(['REGION', 'MONTH', 'YEAR', 'AGE-GROUP', 'SEX', 'COD', 'COUNT'], dtype='object')

In [8]:
# Add region code
region_name = 'REGION'
climate_brussels[region_name] = 4000
climate_antwerp[region_name] = 2000
climate_dinant[region_name] = 3000

In [9]:
# Concatenate dataframes from cities
df_concat = pd.DataFrame().append([climate_brussels, climate_antwerp, climate_dinant])

# Concatenate dataframes from countries
df_countries = pd.DataFrame().append(df_countries)

In [10]:
# Rename date columns
date_dict = {'YEAR': 'YEAR', 'MO': 'MONTH', 'DY': 'DAY'}
df_concat.rename(columns = date_dict, inplace = True)
df_countries.rename(columns = date_dict, inplace = True)

In [11]:
# Create features for multi-countries case
class cleaner_climate_countries(cleaner_climate):  

    def monthly_stat(self):
        df_results = pd.DataFrame()
        df_groupby = self.climate_city.groupby(self.key1)
        # Do some features transformation here, add extra columns if needed
        df_results['TEMP_MEAN'] = df_groupby['T2M_MAX'].mean() # Mean of daily temperature
        return df_results

In [12]:
# Create features
Cleaner = cleaner_climate(df_concat, reference_dist, key = ['REGION', 'YEAR', 'MONTH'])
climate = Cleaner.featurize()

# Create features
Cleaner = cleaner_climate_countries(df_countries, reference_dist_multicountries, key = ['COUNTRY', 'YEAR', 'MONTH'])
climate_countries= Cleaner.featurize()

In [13]:
# Merge with the mortality dataset
mortality_merged = mortality.merge(climate, how = 'left', on = ['REGION', 'YEAR', 'MONTH'])

In [14]:
# Export the mortality dataset
mortality_merged.to_csv('../data/mortality_merged.csv', index = False)

# Export the european countries dataset
climate_countries.to_csv('../data/europe_heatdays.csv', index = False) # Export to data folder for AWS S3 bucket
climate_countries.to_csv('./mda_app/europe_heatdays.csv', index = False) # Export to the directory for the App